# EnigmaNet: A Neural Network Framework for Eplipesy Classification
### This is a script that uses Keras to predict epilepsy outcomes

In [2]:
import enigma

In [3]:
net = enigma.enigmanet(path='/Users/sid/Documents/Projects/Enigma-ML/Dataset/T1/all.csv',
                 classcol='Dx',
                 sitecol='Site',
                 dbegin='ICV',
                 dend='R_insula_surfavg',
                 fillmissing=True,
                 harmonize=True,
                 crange=['Site', 'Sex'],
                 batch='Site',
                 discrete=['Dx', 'Sex'],
                 continuous=['Age'],
                 scale=True,
                 splitdata=0.10)

Found classes: [1 0]


In [4]:
net.transformdata()

...found 2 classes across 16 sites
...filling missing data with class means
Creating design matrix..
Standardizing data across features..
Fitting L/S model and finding priors..
Finding parametric adjustments..
Final adjustment of data..
